# Overview

We will go through a complete 4-step workflow of the tool use

1. Send the user query to the LLM and LLM decides whether to use tool
2. If using tool, LLM will send back a tool use message including the too name and the tool input
3. On user side, the corresponding tool will be called and the tool will return the result
4. The LLM will then see the tool output and continue the conversation


Tool use is a really general enhancement for LLM. For example, RAG can be seen as a tool. 




In [1]:
import wikipedia
from anthropic import Anthropic

client = Anthropic()

In [2]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

In [3]:
def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

In [6]:
model = "claude-3-7-sonnet-20250219"

In [16]:
import re

def extract_answers(text):
    """
    Extracts all text between <answer> and </answer> tags.
    
    Args:
        text (str): Input string containing <answer> tags.
    
    Returns:
        List[str]: A list of extracted text between the tags.
    """
    pattern = r"<answer>(.*?)</answer>"
    ans = re.findall(pattern, text, re.DOTALL)
    return " ".join(ans) if ans else ""

In [17]:
def answer_question(question):
    system_prompt = """
    You will be asked a question by the user. 
    If answering the question requires data you were not trained on, you can use the get_article tool to get the contents of a recent wikipedia article about the topic. 
    If you can answer the question without needing to get more information, please do so. 
    Only call the tool when needed. 
    """
    prompt = f"""
    Answer the following question <question>{question}</question>
    When you can answer the question, keep your answer as short as possible and enclose it in <answer> tags
    """
    messages = [{"role": "user", "content": prompt}]

    response = client.messages.create(
        model=model,
        system=system_prompt, 
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )
    
    print(f"Claude's response: {response}")
    
    # Claude might generate multiple tool use messages. Use a while loop to handle all of them
    while(response.stop_reason == "tool_use"):
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": response.content})

        if tool_name == "get_article":
            search_term = tool_input["search_term"]
            print(f"Claude wants to get an article for {search_term}")
            wiki_result = get_article(search_term) #get wikipedia article content
            #construct our tool_result message
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": wiki_result
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model=model,
                system=system_prompt, 
                messages=messages,
                max_tokens=1000,
                tools=[article_search_tool]
            )
    print("Claude's final answer:")
    print(extract_answers(response.content[0].text))

In [18]:
question = "How many Oscars does Christopher Nolan have?  Does he have more than the number of Emmy's that Ben Stiller has?"
answer_question(question)

Claude's response: Message(id='msg_01TBZf9ZfLSA8HjH3Swt6eNj', content=[TextBlock(citations=None, text="I'll need to check on Christopher Nolan's Oscar wins and Ben Stiller's Emmy wins to answer this question accurately. Let me look up the information.", type='text'), ToolUseBlock(id='toolu_01HD9cUPe2iYU5dqnGZaJPQp', input={'search_term': 'Christopher Nolan'}, name='get_article', type='tool_use')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=546, output_tokens=93, server_tool_use=None, service_tier='standard'))
Claude wants to get an article for Christopher Nolan
Claude wants to get an article for Ben Stiller
Claude's final answer:
Christopher Nolan has two Academy Awards (Oscars), which he won for Best Director and Best Picture for "Oppenheimer" (2023). Ben Stiller has one Emmy Award, which he won for "Outstanding Writing for a Variety